In [3]:
'''
Notebook to Calculate Baseline using a simple average predictor.
'''
import numpy as np
import pandas as pd
import pickle

In [132]:
'''
Utility functions.
'''
# Mapping for data matrix columns.
columns = { 'x' : 0,
            'y' : 1,
            'region' : 2,
            't' : 3, 
            'count' : 4 } 

# Author: Alex Wang -- Sets NaN to average.
def normalize_features(X_train):
    mean_X_train = np.nanmean(X_train, 0)
    for i in xrange(np.shape(X_train)[1]):
        col = X_train[:,i]
        col[ np.isnan(col) ] = mean_X_train[i]
    std_X_train = np.std(X_train, 0)
    std_X_train[ std_X_train == 0 ] = 1
    X_train_normalized = (X_train - mean_X_train) / std_X_train
    return X_train_normalized

def rmse(predict, true):
    # Returns the root mean squared error.
    return np.sqrt(1.0/np.shape(predict)[0] * np.sum(np.square(predict - true)))

def randomSplit(X, split_size):
    # Randomly splits the data.
    np.random.shuffle(X)
    break_pt = int(split_size * np.shape(X)[0])
    return X[:break_pt,:], X[break_pt:,:]

def splitLastN(X, t):
    # Splits the X data matrix into historical data and data for the 
    # last t time steps.
    times = np.unique(X[:, columns['t']])
    lowBound = np.sort(times)[len(times) - t]
    selected = X[:, columns['t']] <= lowBound
    return X[selected,:], X[~selected,:]

def buckets(series, n):
    # Takes a series and returns an array mapping each element to
    # one of n buckets.
    mi, ma = series.min(), series.max()
    buckets = np.linspace(mi, ma, n + 1)
    
    res = np.zeros(len(series))
    try:
        array = series.values
    except AttributeError:
        array = series
    if np.isnan(array).any():
        print "Error! NaN values found in series!"
    for i in xrange(n):
        res[(buckets[i] <= array) & (array < buckets[i+1])] = i
    return res.astype(int)

def createSimplePartitions(data, n):
    # Returns a partitioned version of data into nxn regions!
    data['xRegion'] = buckets(data.Latitude, n).astype(int)
    data['yRegion'] = buckets(data.Longitude, n).astype(int)
    data['Region'] = n * data.xRegion + data.yRegion

    return data

    
def extractRegionsFromMatrix(X_data, n):
    # Does the same thing as extractDataMatrix, but as input takes in
    # a matrix with just the latitude, longitude coordinates, time periods.
    xRegion = buckets(X_data[:, 0], n)
    yRegion = buckets(X_data[:, 1], n)
    Region = n * xRegion + yRegion
    
    xRegions = np.unique(xRegion).astype(int)
    yRegions = np.unique(yRegion).astype(int)
    regions = np.unique(Region).astype(int)
    months = np.unique(X_data[:,2])
    num_columns = 5
    num_rows = len(regions) * len(months)
    X_res = np.zeros((num_rows, num_columns))
    el = 0
    for x in xRegions:
        for y in yRegions:
            for month in months:
                reg = n * x + y
                count = len(X_data[ (Region == reg) &
                                    (X_data[:, 2] == month)])
                if count > 0:
                    X_res[el, :] = np.array([x,y,reg, month, count])
                    el += 1
            
    # Convert data to right shape
    X_res = X_res.astype(int)
    if el < X_res.shape[0]:
        print "Removing empty values from our data!"
        print "Rows before: {}".format(X_res.shape[0])
        X_res = X_res[~np.all(X_res == 0, axis=1)]
        print "Rows after: {}".format(X_res.shape[0])
    
    return X_res

def createDataMatrix(data):
    '''
    Transforms a panda dataframe into latitude longitude time period matrix
    record of crimes.
    '''
    X = np.zeros((len(data), 3))
    X[:, 0] = data.Latitude.values.astype(float)
    X[:, 1] = data.Longitude.values.astype(float)
    X[:, 2] = data.TimeFeature.values.astype(int)
    
    return X

def extractDataMatrix(data, n):
    # Creates a NxD data matrix from the given data set.
    # data must contains xRegion, yRegion, Region, and TimeFeature columns.
    # 0 -> xRegion
    # 1 -> yRegion
    # 2 -> Region
    # 3 -> Month
    # 4 -> Count
    # The data is NOT normalized!
    # Returns the data as well as a dictionary mapping column names
    # to indeces.
    partData = createSimplePartitions(data, n)
    regions = partData.Region.unique()
    xRegions = partData.xRegion.unique()
    yRegions = partData.yRegion.unique()
    months = partData.TimeFeature.unique()
    num_columns = 5
    num_rows = len(regions) * len(months)
    X_data = np.zeros((num_rows, num_columns))
    el = 0
    for x in xRegions:
        for y in yRegions:
            for month in months:
                count = len(data[ (data.xRegion == x) &
                                  (data.yRegion == y) &
                                  (data.TimeFeature == month)])
                if count > 0:
                    X_data[el, :] = np.array([x,y,n*x + y, month, count])
                    el += 1
            
    if el < X_data.shape[0]:
        print "Removing empty values from our data!"
        print "Rows before: {}".format(X_data.shape[0])
        X_data = X_data[~np.all(X_data == 0, axis=1)]
        print "Rows after: {}".format(X_data.shape[0])
        
    return X_data.astype(int)

In [136]:
'''
More utility functions. 
'''
# Note: If a data point does not exist, it is assumed to be 0.
def averagePredictions(X_train, nRegions, tMax):
    averages = np.zeros(nRegions)
    for region in xrange(nRegions):
        averages[region] = X_train[
            X_train[:, columns['region']] == region,
            columns['count']].sum() / float(tMax)
    return averages

def createHeatMap(X):
    '''
    Given a data set, creates a heatmap of it based on x,y coordinates.
    Ignore the temporal feature. You should subset the data before passing
    it into this function if you'd like a heatmap for a specific time period.
    '''
    n = X[:, columns['x']].astype(int).max()
    m = X[:, columns['y']].astype(int).max()
    heatmap = np.zeros((n,m))
    for i in xrange(n):
        for j in xrange(m):
            total = X[(X[:, columns['x']] == i) & 
                      (X[:, columns['y']] == j), columns['count']].sum()
            if total > 0:
                heatmap[i,j] = total
                
    return heatmap

In [146]:
# Given a value of n:
# 0. Normalize the data (if set to True)
# 1. Partition the data
# 2. Split into Train/Test, where test has lastN time feats.
#    Options: 'random', 'last'
#    splitRatio specifies the ratio of results to keep for testing.
#    testPeriods specifies the number of time periods to test
# 3. Train the averages
# 4. Test on the hold-out
# 5. Calculate RMSE
def averageModel(n, X_data, normalize = False, splitMethod = 'random', splitRatio = 0.1, testPeriods = 12):
    if normalize:
        X_data = normalize_features(X_data)
        print "Normalized data features!"
        sys.stdout.flush()
        
    # Returns an array indexed by region with the average over the 
    # training set for each region.
    tMax = X_data[:, columns['t']].max()
    nRegions = X_data[:, columns['region']].max() + 1
    
    # Now we can split the data!
    if splitMethod == 'random':
        X_train, X_test = randomSplit(X_data, splitRatio)
    elif splitMethod == 'last':
        X_train, X_test = splitLastN(X_data, testPeriods)
    else:
        raise Exception("splitMethod {} unsupported".format(splitMethod))
    
    print "Training model..."
    sys.stdout.flush()
    # Now use training data to calculate averages
    model = averagePredictions(X_train, nRegions, tMax)
    print "Averages Model:"
    print model
    sys.stdout.flush()
    
    # Generate predictions vector
    predict = model[X_test[:, columns['region']]]
    true = X_test[:, columns['count']]
    
    # Create distributions, and calculate the RMSE of the distribution
    predict_dist = predict / float(np.sum(predict))
    true_dist = true / float(np.sum(true))
    
    print "Calculating RMSE..."
    sys.stdout.flush()
    
    return rmse(predict_dist, true_dist)

In [126]:
# Let's make a plot for some values of N to see if the data works out...
sfdata_file = '../../cs281_data/large_data/sfclean.pk'
with open(sfdata_file) as fp:
    sfdata = pickle.load(fp)
    # For sfdata, need to remove outliers
    sfdata = sfdata[-120 > sfdata.Longitude][sfdata.Longitude > (-130)]
    sfdata = sfdata[sfdata.Latitude > 37][sfdata.Latitude < 40]

In [148]:
import sys

# Create the data matrix for SanFrancisco
X_data = createDataMatrix(sfdata)

testN = range(2,10) + range(10,20,5)
rmses = []
for n in testN:
    print "n = {}".format(n)
    # Extracting more efficiently!
    %time X_region = extractRegionsFromMatrix(X_data, n)
    # print X_data.dtype
    print "Partitioned data..."
    sys.stdout.flush()
    rmse_random = averageModel(n, X_region)
    print "Random RMSE: {}".format(rmse_random)
    sys.stdout.flush()
    rmse_last = averageModel(n, X_region, splitMethod='last')
    rmses.append((rmse_random, rmse_last))
    print "Last RMSE: {}".format(rmse_last)
    sys.stdout.flush()

n = 2
CPU times: user 7.68 s, sys: 0 ns, total: 7.68 s
Wall time: 7.68 s
Partitioned data...
Training model...
Averages Model:
[ 134.86363636  353.5          87.86363636  628.05844156]
Calculating RMSE...
Random RMSE: 0.000240861389785
Training model...
Averages Model:
[ 1157.28571429  2909.2987013    994.33766234  5939.02597403]
Calculating RMSE...
Last RMSE: 0.00290841107286
n = 3
CPU times: user 16 s, sys: 0 ns, total: 16 s
Wall time: 16 s
Partitioned data...
Training model...
Averages Model:
[  22.85064935  114.05194805   69.66233766   66.5         298.5
  306.31818182    5.66883117   60.42857143  308.96753247]
Calculating RMSE...
Random RMSE: 0.000324627473434
Training model...
Averages Model:
[  360.01948052   986.3961039    977.35714286   601.33116883  2663.23376623
  2097.98051948    42.88961039  1399.98051948  1870.75974026]
Calculating RMSE...
Last RMSE: 0.00134040127748
n = 4
CPU times: user 28.2 s, sys: 0 ns, total: 28.2 s
Wall time: 28.2 s
Partitioned data...
Training mode

KeyboardInterrupt: 

In [ ]:
# Calculat the RMSE for n = 15
n = 15
X_data = extractDataMatrix(sfdata, n)
rmse_random15 = averageModel(n, X_data)
rmse_last15 = averageModel(n, X_data, splitMethod='last')

In [102]:
rmses, rmse_last15, rmse_random15

([(3251.3638018497254, 515.77474251496869),
  (1447.171064391375, 215.41729545008857),
  (1289.3316464751988, 167.83815175812575),
  (809.42313953565497, 112.07265100677968),
  (522.33102592997, 88.853815771321806),
  (483.67467985515685, 85.632104612382335),
  (411.730711190791, 70.962471993066828),
  (301.01978243896173, 53.300374089523864),
  (260.72621104254262, 57.288523402690132),
  (133.00384412934449, 29.691957633717099),
  (84.002157945116792, 25.657276412496376),
  (42.734686731225423, 13.973661083494004),
  (27.176216198750392, 9.7661827365748941),
  (19.958992915729283, 7.5008050474014656),
  (15.986312894185623, 6.0995323817263838)],
 29.691957633717099,
 133.00384412934449)

In [101]:
rmses = rmses[:9] + [(rmse_random15, rmse_last15)] + rmses[9:]

In [ ]:
# Generate a plot of the RMSEs that we calculated
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("darkgrid")

In [111]:
x = testN[:len(rmses)]
y1,y2 = zip(*rmses)
line1 = plt.plot(x, y1, label="Final 12 Time Units")
line2 = plt.plot(x, y2, label="Random Sample Data")
plt.title('Baseline Predictions for San Francisco')
plt.xlabel('Dimension of Grid')
plt.ylabel('Root Mean Square Error')
# plt.legend()
plt.show()